<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/NER/ner1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install transformers datasets evaluate seqeval

1. 載入相關套件

In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset

2使用數據集

In [26]:
ner_datasets = load_dataset("peoples_daily_ner",cache_dir='./data',trust_remote_code=True)

In [ ]:
#查看第一筆資料
ner_datasets['train'][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [27]:
#查看ner_tags,5,6,所代表的意思
ner_datasets['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [28]:
#取得label_list
label_list = ner_datasets['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

3數據預處理

In [29]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

In [30]:
#不正確,這是以字為單位,而不是以句子為單位
tokenizer(ner_datasets['train'][0]["tokens"])

{'input_ids': [[101, 3862, 102], [101, 7157, 102], [101, 3683, 102], [101, 6612, 102], [101, 1765, 102], [101, 4157, 102], [101, 1762, 102], [101, 1336, 102], [101, 7305, 102], [101, 680, 102], [101, 7032, 102], [101, 7305, 102], [101, 722, 102], [101, 7313, 102], [101, 4638, 102], [101, 3862, 102], [101, 1818, 102], [101, 511, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [31]:
tokenizer(ner_datasets['train'][0]['tokens'], is_split_into_words=True) #必需使用is_split_into_words

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [32]:
#英文的拆字,注意數量(10673, 12865, 12921, 8181, 8681)
res = tokenizer('interesting word')
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [33]:
# 0,0,0,0代表第1個字
# 1代表第2個字
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [34]:
from pprint import pprint
def process_batch(examples):
    tokenized_examples = tokenizer(examples['tokens'], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels

    return tokenized_examples


In [35]:
tokenized_datasets = ner_datasets.map(process_batch,batched=True)
tokenized_datasets['train'][0]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  3862,
  7157,
  3683,
  6612,
  1765,
  4157,
  1762,
  1336,
  7305,
  680,
  7032,
  7305,
  722,
  7313,
  4638,
  3862,
  1818,
  511,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}

In [36]:
from pprint import pprint
pprint(tokenized_datasets['train'][0])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'id': '0',
 'input_ids': [101,
               3862,
               7157,
               3683,
               6612,
               1765,
               4157,
               1762,
               1336,
               7305,
               680,
               7032,
               7305,
               722,
               7313,
               4638,
               3862,
               1818,
               511,
               102],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['海',
            '钓',
            '比',
            '赛',
            '地',
            '点',
            '在',
            '厦',
            '门',
            '与',
            '金',
            '门',
            '之',
            '间',
            '的',
       

4建立模型

In [37]:
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-base-chinese",num_labels=len(label_list)) #預設是2,現在是7個

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
model.num_labels

7

In [39]:
seqeval = evaluate.load('seqeval')
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [40]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    true_predictions =[
        [label_list[p] for p,l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "f1": result["overall_f1"]
    }

配置訓練參數

In [41]:
args = TrainingArguments(
    output_dir = "models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps = 50,
    num_train_epochs=3,
    report_to='none'
)

建立訓練器

In [42]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=eval_metric,
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
)

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.027400,0.020366,0.950964
2,0.012700,0.017351,0.959242
3,0.006300,0.018629,0.960175


TrainOutput(global_step=981, training_loss=0.02864584076112928, metrics={'train_runtime': 1237.5982, 'train_samples_per_second': 50.578, 'train_steps_per_second': 0.793, 'total_flos': 3940951205762142.0, 'train_loss': 0.02864584076112928, 'epoch': 3.0})

評估模型

In [44]:
trainer.evaluate(eval_dataset=tokenized_datasets['test'])

{'eval_loss': 0.02187521383166313,
 'eval_f1': 0.9508438253415484,
 'eval_runtime': 36.0562,
 'eval_samples_per_second': 128.605,
 'eval_steps_per_second': 1.026,
 'epoch': 3.0}

上傳模型

In [45]:
from huggingface_hub import login
login()

In [46]:
trainer.push_to_hub("roberthsu2003") #由於有設./checkpoints,所以自動產生checkpoints的repo,也會自動上傳評估至repo
#同時要上傳tokenizer
model_name = "roberthsu2003/models_for_ner"
tokenizer.push_to_hub(model_name)


training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/407M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/roberthsu2003/models_for_ner/commit/379a8f146029422b207f7edec8ece692fffef861', commit_message='Upload tokenizer', commit_description='', oid='379a8f146029422b207f7edec8ece692fffef861', pr_url=None, repo_url=RepoUrl('https://huggingface.co/roberthsu2003/models_for_ner', endpoint='https://huggingface.co', repo_type='model', repo_id='roberthsu2003/models_for_ner'), pr_revision=None, pr_num=None)

下載模型
模型預測
pipeline()->使用NER的官方說明
https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TokenClassificationPipeline

In [18]:
from transformers import pipeline,AutoModelForTokenClassification,AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')
tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
ner_pipe("徐國堂在台北上班")

Device set to use cpu


[{'entity_group': 'LABEL_1',
  'score': np.float32(0.99952245),
  'word': '徐',
  'start': 0,
  'end': 1},
 {'entity_group': 'LABEL_2',
  'score': np.float32(0.9994726),
  'word': '國 堂',
  'start': 1,
  'end': 3},
 {'entity_group': 'LABEL_0',
  'score': np.float32(0.99968326),
  'word': '在',
  'start': 3,
  'end': 4},
 {'entity_group': 'LABEL_5',
  'score': np.float32(0.99884737),
  'word': '台',
  'start': 4,
  'end': 5},
 {'entity_group': 'LABEL_6',
  'score': np.float32(0.99812204),
  'word': '北',
  'start': 5,
  'end': 6},
 {'entity_group': 'LABEL_0',
  'score': np.float32(0.99980557),
  'word': '上 班',
  'start': 6,
  'end': 8}]

In [ ]:
#顯示
print(model.config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6'}


In [24]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config.id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC'}

In [34]:
from huggingface_hub import login
login()


In [36]:
model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')

tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')
model_name = "roberthsu2003/models_for_ner"
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/roberthsu2003/models_for_ner/commit/a282114ddb128460da86f40dfa24c65023dee15d', commit_message='Upload tokenizer', commit_description='', oid='a282114ddb128460da86f40dfa24c65023dee15d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/roberthsu2003/models_for_ner', endpoint='https://huggingface.co', repo_type='model', repo_id='roberthsu2003/models_for_ner'), pr_revision=None, pr_num=None)

In [25]:
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
ner_pipe("徐國堂在台北上班")

Device set to use cpu


[{'entity_group': 'PER',
  'score': np.float32(0.99948925),
  'word': '徐 國 堂',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': np.float32(0.99848473),
  'word': '台 北',
  'start': 4,
  'end': 6}]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')
tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
inputs = "徐國堂在台北上班" 
res = ner_pipe(inputs)

ner_result = {}
for r in res:
  if r["entity_group"] not in ner_datasets:
    ner_result[r['entity_group']] = []
  ner_result[r['entity_group']].append(inputs[r['start']:r['end']])
ner_result

Device set to use cpu


{'PER': ['徐國堂'], 'LOC': ['台北']}

In [ ]:
#模型使用方式,pipline的使用方式
ner_pipe = pipeline("token-classification", model='roberthsu2003/models_for_ner',aggregation_strategy="simple")
inputs = "徐國堂在台北上班" 
res = ner_pipe(inputs)

ner_result = {}
for r in res:
  if r["entity_group"] not in ner_datasets:
    ner_result[r['entity_group']] = []
  ner_result[r['entity_group']].append(inputs[r['start']:r['end']])
ner_result

Device set to use cpu


{'PER': ['徐國堂'], 'LOC': ['台北']}

In [ ]:
#使用model,tokenizer的使用方法
from transformers import AutoModelForTokenClassification, AutoTokenizer
import numpy as np

# Load the pre-trained model and tokenizer
model = AutoModelForTokenClassification.from_pretrained('roberthsu2003/models_for_ner')
tokenizer = AutoTokenizer.from_pretrained('roberthsu2003/models_for_ner')

# The label mapping (you might need to adjust this based on your training)
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

def predict_ner(text):
    """Predicts NER tags for a given text using the loaded model."""
    # Encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # Get model predictions
    outputs = model(**inputs)
    predictions = np.argmax(outputs.logits.detach().numpy(), axis=-1)
    
    # Get the word IDs from the encoded inputs
    # This is the key change - word_ids() is a method on the encoding result, not the tokenizer itself
    word_ids = inputs.word_ids(batch_index=0)
    
    pred_tags = []
    for word_id, pred in zip(word_ids, predictions[0]):
        if word_id is None:
            continue  # Skip special tokens
        pred_tags.append(label_list[pred])

    return pred_tags

#To get the entities, you'll need to group consecutive non-O tags:

def get_entities(tags):
    """Groups consecutive NER tags to extract entities."""
    entities = []
    start_index = -1
    current_entity_type = None
    for i, tag in enumerate(tags):
        if tag != 'O':
            if start_index == -1:
                start_index = i
                current_entity_type = tag[2:] # Extract entity type (e.g., PER, LOC, ORG)
        else: #tag == 'O'
            if start_index != -1:
                entities.append((start_index, i, current_entity_type))
                start_index = -1
                current_entity_type = None
    if start_index != -1:
        entities.append((start_index, len(tags), current_entity_type))
    return entities

# Example usage:
text = "徐國堂在台北上班"
ner_tags = predict_ner(text)
print(f"Text: {text}")
#==output==
#Text: 徐國堂在台北上班
print(f"NER Tags: {ner_tags}")
#===output==
#NER Tags: ['B-PER', 'I-PER', 'I-PER', 'O', 'B-LOC', 'I-LOC', 'O', 'O']


entities = get_entities(ner_tags)
word_tokens = tokenizer.tokenize(text)  # Tokenize to get individual words
print(f"Entities:")
for start, end, entity_type in entities:
    entity_text = "".join(word_tokens[start:end])
    print(f"- {entity_text}: {entity_type}")

#==output==
#Entities:
#- 徐國堂: PER
#- 台北: LOC


Text: 徐國堂在台北上班
NER Tags: ['B-PER', 'I-PER', 'I-PER', 'O', 'B-LOC', 'I-LOC', 'O', 'O']
Entities:
- 徐國堂: PER
- 台北: LOC
